In [2]:
import pandas as pd
import numpy as np
import nbimporter
import feature_engineering
from datetime import datetime, timedelta
import glob
import os

In [3]:
folder_path = r"Stromverbrauch\Viertel"
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

stromverbrauch_list = []

for file in csv_files:
    #folder_path = r"C:\Users\Isabell\EMSI\emsi_ml\Stromverbrauch\Viertel\Realisierter_Stromverbrauch_201506070000_201506100000_Viertelstunde.csv"
    stromverbrauch = pd.read_csv(file, delimiter = ";")
    stromverbrauch["Datum von"] = pd.to_datetime(stromverbrauch["Datum von"], format='%d.%m.%Y %H:%M')
    stromverbrauch_list.append(stromverbrauch)


df_tuples = [(stromverbrauch["Datum von"].iloc[0], stromverbrauch) for stromverbrauch in stromverbrauch_list ]

# Sort the list of tuples by the earliest date
df_tuples_sorted = sorted(df_tuples, key=lambda x: x[0])

dfs_by_year = {}

# Iterate over the list of tuples
for dt, df in df_tuples_sorted:  # Correctly unpack: dt is the date, df is the DataFrame
    year = dt.year  # Extract the year from the date
    
    if year not in dfs_by_year:
        dfs_by_year[year] = df
    else:
        # Merge the DataFrame with the existing one for the same year
        dfs_by_year[year] = pd.concat([dfs_by_year[year], df], ignore_index=True)

# Extract the sorted DataFrames into a list
sorted_df_list = [df_tuple[1] for df_tuple in df_tuples_sorted]

for year, merged_df in dfs_by_year.items():
    print(f"Year: {year}")
    print(merged_df)



Year: 2015
               Datum von         Datum bis  \
0    2015-06-07 00:00:00  07.06.2015 00:15   
1    2015-06-07 00:15:00  07.06.2015 00:30   
2    2015-06-07 00:30:00  07.06.2015 00:45   
3    2015-06-07 00:45:00  07.06.2015 01:00   
4    2015-06-07 01:00:00  07.06.2015 01:15   
...                  ...               ...   
3163 2015-07-12 22:45:00  12.07.2015 23:00   
3164 2015-07-12 23:00:00  12.07.2015 23:15   
3165 2015-07-12 23:15:00  12.07.2015 23:30   
3166 2015-07-12 23:30:00  12.07.2015 23:45   
3167 2015-07-12 23:45:00  13.07.2015 00:00   

     Gesamt (Netzlast) [MWh] Originalauflösungen  
0                                      10.419,00  
1                                      10.234,50  
2                                      10.000,00  
3                                       9.836,75  
4                                       9.703,00  
...                                          ...  
3163                                   11.883,50  
3164                        

In [15]:
folder_path = r"Spielplan"
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

spielplan_list = []

for file in csv_files:
    spielplan = pd.read_csv(file, delimiter = ";")

    spielplan["DateTime"] = spielplan["Datum"] + " " + spielplan["Uhrzeit (MESZ)"]
    try:
        spielplan["DateTime"] = pd.to_datetime(spielplan["DateTime"], format='%d. %b %y %H:%M')
    except:
        spielplan["DateTime"] = pd.to_datetime(spielplan["DateTime"], format='%d.%m.%Y %H:%M')

    if "EM" in file:
        spielplan["Liga"] = "EM"
    elif "WM" in file:
        spielplan["Liga"] = "WM"



    #print(spielplan.columns)
    #spielplan["Datum"] = pd.to_datetime(spielplan["Datum"], format = "%d. %b %y")

   
    #spielplan.set_index("DateTime", inplace = True)

    spielplan = spielplan.loc[:, ["Land1", "Land2", "Runde", "Liga", "DateTime"]]
    #print(spielplan)
    grouped = spielplan.groupby("DateTime")
    #print(grouped)
    # Initialize a new DataFrame to hold the results
    results = pd.DataFrame()
    
    #row = {}
    for name, group in grouped:
        #print(name)
        #print(group)
        if len(group) == 1:
            results = pd.concat([results, group])
            #print(results)
            #results["DateTime"] = name
            #results["Land3"] = np.nan
            #results["Land4"] = np.nan
        else:
            land1_value = group.iloc[0]['Land1']
            land2_value = group.iloc[0]['Land2']
            
            # Update results DataFrame (Example: adding values from the second row to a new DataFrame)
            new_row = {
                "DateTime" : name,
                'Land1': land1_value,
                'Land2': land2_value,
                "Runde" : group.iloc[1]["Runde"],
                "Liga": group.iloc[1]["Liga"],
                'Land3': group.iloc[1]['Land1'],
                'Land4': group.iloc[1]['Land2'],
                
            }

            new_df = pd.DataFrame([new_row])
            #print(new_df)
            #new_df.set_index("DateTime")
            results = pd.concat([results, new_df], ignore_index = True)
    results.set_index("DateTime")
    
    spielplan_list.append(results)

        #print(results["Runde"].unique())
        #print(results)
#print(lenspielplan_list)
print(len(spielplan_list))
    

5


In [140]:

#index_counter = 0
counter_limit = len(spielplan_list) - 1

merged_df_list = []

for key,value in dfs_by_year.items():
    found_item = False
    index_counter = 0
    while found_item == False:
        spielplan = spielplan_list[index_counter]
        #rint(spielplan)
        spielplan_name_list = spielplan.columns.tolist()

        if key == spielplan["DateTime"].iloc[0].year:
            merged_df = pd.merge(value, spielplan, left_on="Datum von", right_on='DateTime', how='outer')
            
            for index, row in spielplan.iterrows():
                start_time = row["DateTime"]
                print(start_time)
                until_time = row["DateTime"] + timedelta(minutes=105)
                #print(until_time)
                fill_condition = (merged_df['Datum von']>= start_time) & (merged_df['Datum von'] <= until_time)
                merged_df.loc[fill_condition,spielplan_name_list] = merged_df.loc[fill_condition, spielplan_name_list].fillna(method='ffill')


            merged_df_list.append(merged_df)
            found_item = True
        elif index_counter == counter_limit:
            found_item = True
        else:
            index_counter += 1
    
df = merged_df_list[0]

#df = df.loc[2965:2980,["Datum von", "Land1", "Gesamt (Netzlast) [MWh] Originalauflösungen"]]

print(df)

#print(df["Land1"].unique())
#df.set_index("Datum von", inplace = True)
#data = df.loc["2016-06-11"]            
#print(data)
#merged_df_list.append(merged_df)

2016-06-10 21:00:00
2016-06-11 15:00:00
2016-06-11 18:00:00
2016-06-11 21:00:00
2016-06-12 15:00:00
2016-06-12 18:00:00
2016-06-12 21:00:00
2016-06-13 15:00:00
2016-06-13 18:00:00
2016-06-13 21:00:00
2016-06-14 18:00:00
2016-06-14 21:00:00
2016-06-15 15:00:00
2016-06-15 18:00:00
2016-06-15 21:00:00
2016-06-16 15:00:00
2016-06-16 18:00:00
2016-06-16 21:00:00
2016-06-17 15:00:00
2016-06-17 18:00:00
2016-06-17 21:00:00
2016-06-18 15:00:00
2016-06-18 18:00:00
2016-06-18 21:00:00
2016-06-19 21:00:00
2018-06-14 17:00:00
2018-06-15 14:00:00
2018-06-15 17:00:00
2018-06-15 20:00:00
2018-06-16 12:00:00
2018-06-16 15:00:00
2018-06-16 18:00:00
2018-06-16 21:00:00
2018-06-17 14:00:00
2018-06-17 17:00:00


C:\Users\Isabell\AppData\Local\Temp\ipykernel_12340\4201209102.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.loc[fill_condition,spielplan_name_list] = merged_df.loc[fill_condition, spielplan_name_list].fillna(method='ffill')
C:\Users\Isabell\AppData\Local\Temp\ipykernel_12340\4201209102.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.loc[fill_condition,spielplan_name_list] = merged_df.loc[fill_condition, spielplan_name_list].fillna(method='ffill')
C:\Users\Isabell\AppData\Local\Temp\ipykernel_12340\4201209102.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.loc[fill_condition,spielplan_name_list] = merged_df.loc[fill_condition, spielplan_name_list].fillna(method='ffill'

2018-06-17 20:00:00
2018-06-18 14:00:00
2018-06-18 17:00:00
2018-06-18 20:00:00
2018-06-19 14:00:00
2018-06-19 17:00:00
2018-06-19 20:00:00
2018-06-20 14:00:00
2018-06-20 17:00:00
2018-06-20 20:00:00
2018-06-21 14:00:00
2018-06-21 17:00:00
2018-06-21 20:00:00
2018-06-22 14:00:00
2018-06-22 17:00:00
2018-06-22 20:00:00
2018-06-23 14:00:00
2018-06-23 17:00:00
2018-06-23 20:00:00
2018-06-24 14:00:00
2018-06-24 17:00:00
2018-06-24 20:00:00
2018-06-25 16:00:00
2021-06-11 21:00:00
2021-06-12 15:00:00
2021-06-12 18:00:00
2021-06-12 21:00:00
2021-06-13 15:00:00
2021-06-13 18:00:00
2021-06-13 21:00:00
2021-06-14 15:00:00
2021-06-14 18:00:00
2021-06-14 21:00:00
2021-06-15 18:00:00
2021-06-15 21:00:00
2021-06-16 15:00:00
2021-06-16 18:00:00
2021-06-16 21:00:00
2021-06-17 15:00:00
2021-06-17 18:00:00
2021-06-17 21:00:00
2021-06-18 15:00:00
2021-06-18 18:00:00
2021-06-18 21:00:00
2021-06-19 15:00:00
2021-06-19 18:00:00
2021-06-19 21:00:00
2021-06-20 18:00:00
               Datum von         Datum b

C:\Users\Isabell\AppData\Local\Temp\ipykernel_12340\4201209102.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.loc[fill_condition,spielplan_name_list] = merged_df.loc[fill_condition, spielplan_name_list].fillna(method='ffill')
C:\Users\Isabell\AppData\Local\Temp\ipykernel_12340\4201209102.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.loc[fill_condition,spielplan_name_list] = merged_df.loc[fill_condition, spielplan_name_list].fillna(method='ffill')
C:\Users\Isabell\AppData\Local\Temp\ipykernel_12340\4201209102.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.loc[fill_condition,spielplan_name_list] = merged_df.loc[fill_condition, spielplan_name_list].fillna(method='ffill'